# H04 MCU Evaluation and Verification Helpers

In [22]:
def set_port(port, baudrate):
    port = serial.Serial(port = port, baudrate=baudrate, timeout=60, write_timeout=None,
                         stopbits=serial.STOPBITS_ONE, bytesize=8)
    
    return port

In [23]:
def send_image(port, image):
    image = np.array(image.flatten(), np.float32)

    if platform == "darwin":
        for b in image:
            port.write(bytes(bytearray(b)))
            time.sleep(0.0005)
    else:
        port.write(bytes(bytearray(image)))

In [24]:
def read_results(port):
    try:
        output = port.read_until(bytes("_end_report_\n\n".encode('utf-8')))
    except serial.SerialTimeoutException:
        print("Read timeout ...")
        return -1, -1
    except Exception as err:
        print("Unknown error:", err)
        return -1, -1

    try:
        output = output.decode('utf-8')
    except UnicodeDecodeError:
        print("UnicodeDecodeError")
        return -1, -1
        
    prediction = np.empty((0))
    benchmark_result = -1
    layers_benchmarks = {}

    lines = iter(output.splitlines())
    
    for line in lines:
        if "Number of inferences" in line:
            number_of_inferences = int(next(lines))
       
        elif "Duration of inferences in" in line:
            benchmark_result = int(next(lines))
            benchmark_result = benchmark_result / number_of_inferences
    
        elif "Class" in line:
            prediction = np.append(prediction, (float.fromhex(next(lines))))
            
        elif "Layer" in line:
            layer_type = str(line[8:]) # skip 'Layer_'
            layers_benchmarks[layer_type] = int(next(lines))
            
        elif not line or "_end_report_" in line or "_start_report_" in line:
            # empty line - skip
            next(lines)

        else:
            print("Unknown serial input:\n", line, next(lines))
    
    # when we didn't read any benchmark results
    # replace the return value with the benchmarking results
    if benchmark_result == -1:
        benchmark_result = layers_benchmarks
    
    return prediction, benchmark_result

In [8]:
def read_raw_results(port):
    try:
        output = port.read_until(bytes("_end_report_\n\n".encode('utf-8')))
    except serial.SerialTimeoutException:
        print("Read timeout ...")
        return -1, -1
    except Exception as err:
        print("Unknown error:", err)
        return -1, -1

    try:
        output = output.decode('utf-8')
    except UnicodeDecodeError:
        print("UnicodeDecodeError")
        return -1, -1
        
    prediction = np.empty((0))
    benchmark_result = -1
    layers_benchmarks = {}

    lines = iter(output.splitlines())
    
    for line in lines:
        print(line, next(lines))
    
    
    return 1

#### Sent images to MCU and log predictions

This function sends the whole normalized testset to the MCU, waits for the predictions and logs them.

In [25]:
def verify_accuracy_on_mcu(port, test=False):

    mcu_predictions = np.empty((0, 10), np.float32)
    benchmark_results = np.empty((0))
    
    # read header and clear queue
    port.read_all()

    if test:
        testset = x_test_normalized[:50]
    else:
        testset = x_test_normalized
        
    for test_image in testset:
        send_image(port, test_image)
        mcu_prediction, benchmark_result = read_results(port)
        mcu_predictions = np.append(mcu_predictions, [mcu_prediction], axis=0)
        benchmark_results = np.append(benchmark_results, benchmark_result)

    return mcu_predictions, benchmark_results

In [1]:
print("Imported helper functions from H04_MCU-Verification.ipynb")

Imported helper functions from H04_MCU-Verification.ipynb
